##  MaskAngleConstraint tester
Note: uses astropy 2.0 and astroplan 0.4.  Numpy is used when possible.

In [134]:
#!/usr/bin/env python3
"""
MaskAngleConstraint tester.
"""

'\nMaskAngleConstraint tester.\n'

In [135]:
import pkg_resources
pkg_resources.require("astropy>=2.0")
pkg_resources.require("astroplan>=0.3")

[astroplan 0.4 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 astropy 2.0.2 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 pytz 2017.2 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 pytest 3.2.1 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 setuptools 36.5.0.post20170921 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages),
 py 1.4.34 (/Users/jdgibson/anaconda/envs/QueueScheduler2_0/lib/python3.6/site-packages)]

In [136]:
# from astroplan import download_IERS_A
# download_IERS_A()

In [137]:
from astroplan import Observer, FixedTarget
from astropy.time import Time, TimeDelta

In [164]:
from astroplan import Constraint
from astroplan import Observer
from astroplan.constraints import _get_altaz, _get_meridian_transit_times, _make_cache_key
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import datetime
import sys
import os

In [139]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# This code is distributed under the terms and conditions
# from the Apache License, Version 2.0
#
# http://opensource.org/licenses/apache2.0.php
#
# This code was inspired by:
#  * http://code.activestate.com/recipes/576638-draft-for-an-sqlite3-based-dbm/
#  * http://code.activestate.com/recipes/526618/

"""
A lightweight wrapper around Python's sqlite3 database, with a dict-like interface
and multi-thread access support::

>>> mydict = SqliteDict('some.db', autocommit=True) # the mapping will be persisted to file `some.db`
>>> mydict['some_key'] = any_picklable_object
>>> print mydict['some_key']
>>> print len(mydict) # etc... all dict functions work

Pickle is used internally to serialize the values. Keys are strings.

If you don't use autocommit (default is no autocommit for performance), then
don't forget to call `mydict.commit()` when done with a transaction.

"""

import sqlite3
import os
import sys
import tempfile
import random
import logging
import traceback

from threading import Thread

try:
    __version__ = __import__('pkg_resources').get_distribution('sqlitedict').version
except:
    __version__ = '?'

major_version = sys.version_info[0]
if major_version < 3:  # py <= 2.x
    if sys.version_info[1] < 5:  # py <= 2.4
        raise ImportError("sqlitedict requires python 2.5 or higher (python 3.3 or higher supported)")

    # necessary to use exec()_ as this would be a SyntaxError in python3.
    # this is an exact port of six.reraise():
    def exec_(_code_, _globs_=None, _locs_=None):
        """Execute code in a namespace."""
        if _globs_ is None:
            frame = sys._getframe(1)
            _globs_ = frame.f_globals
            if _locs_ is None:
                _locs_ = frame.f_locals
            del frame
        elif _locs_ is None:
            _locs_ = _globs_
        exec("""exec _code_ in _globs_, _locs_""")

    exec_("def reraise(tp, value, tb=None):\n"
          "    raise tp, value, tb\n")
else:
    def reraise(tp, value, tb=None):
        if value is None:
            value = tp()
        if value.__traceback__ is not tb:
            raise value.with_traceback(tb)
        raise value

try:
    from cPickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL
except ImportError:
    from pickle import dumps, loads, HIGHEST_PROTOCOL as PICKLE_PROTOCOL

# some Python 3 vs 2 imports
try:
    from collections import UserDict as DictClass
except ImportError:
    from UserDict import DictMixin as DictClass

try:
    from queue import Queue
except ImportError:
    from Queue import Queue


logger = logging.getLogger(__name__)


def open(*args, **kwargs):
    """See documentation of the SqliteDict class."""
    return SqliteDict(*args, **kwargs)


def encode(obj):
    """Serialize an object using pickle to a binary format accepted by SQLite."""
    return sqlite3.Binary(dumps(obj, protocol=PICKLE_PROTOCOL))


def decode(obj):
    """Deserialize objects retrieved from SQLite."""
    return loads(bytes(obj))


class SqliteDict(DictClass):
    VALID_FLAGS = ['c', 'r', 'w', 'n']

    def __init__(self, filename=None, tablename='unnamed', flag='c',
                 autocommit=False, journal_mode="DELETE", encode=encode, decode=decode):
        """
        Initialize a thread-safe sqlite-backed dictionary. The dictionary will
        be a table `tablename` in database file `filename`. A single file (=database)
        may contain multiple tables.

        If no `filename` is given, a random file in temp will be used (and deleted
        from temp once the dict is closed/deleted).

        If you enable `autocommit`, changes will be committed after each operation
        (more inefficient but safer). Otherwise, changes are committed on `self.commit()`,
        `self.clear()` and `self.close()`.

        Set `journal_mode` to 'OFF' if you're experiencing sqlite I/O problems
        or if you need performance and don't care about crash-consistency.

        The `flag` parameter. Exactly one of:
          'c': default mode, open for read/write, creating the db/table if necessary.
          'w': open for r/w, but drop `tablename` contents first (start with empty table)
          'r': open as read-only
          'n': create a new database (erasing any existing tables, not just `tablename`!).

        The `encode` and `decode` parameters are used to customize how the values
        are serialized and deserialized.
        The `encode` parameter must be a function that takes a single Python
        object and returns a serialized representation.
        The `decode` function must be a function that takes the serialized
        representation produced by `encode` and returns a deserialized Python
        object.
        The default is to use pickle.

        """
        self.in_temp = filename is None
        if self.in_temp:
            randpart = hex(random.randint(0, 0xffffff))[2:]
            filename = os.path.join(tempfile.gettempdir(), 'sqldict' + randpart)

        if flag not in SqliteDict.VALID_FLAGS:
            raise RuntimeError("Unrecognized flag: %s" % flag)
        self.flag = flag

        if flag == 'n':
            if os.path.exists(filename):
                os.remove(filename)

        dirname = os.path.dirname(filename)
        if dirname:
            if not os.path.exists(dirname):
                raise RuntimeError('Error! The directory does not exist, %s' % dirname)

        self.filename = filename
        if '"' in tablename:
            raise ValueError('Invalid tablename %r' % tablename)
        self.tablename = tablename
        self.autocommit = autocommit
        self.journal_mode = journal_mode
        self.encode = encode
        self.decode = decode

        logger.info("opening Sqlite table %r in %s" % (tablename, filename))
        MAKE_TABLE = 'CREATE TABLE IF NOT EXISTS "%s" (key TEXT PRIMARY KEY, value BLOB)' % self.tablename
        self.conn = self._new_conn()
        self.conn.execute(MAKE_TABLE)
        self.conn.commit()
        if flag == 'w':
            self.clear()

    def _new_conn(self):
        return SqliteMultithread(self.filename, autocommit=self.autocommit, journal_mode=self.journal_mode)

    def __enter__(self):
        if not hasattr(self, 'conn') or self.conn is None:
            self.conn = self._new_conn()
        return self

    def __exit__(self, *exc_info):
        self.close()

    def __str__(self):
        return "SqliteDict(%s)" % (self.filename)

    def __repr__(self):
        return str(self)  # no need of something complex

    def __len__(self):
        # `select count (*)` is super slow in sqlite (does a linear scan!!)
        # As a result, len() is very slow too once the table size grows beyond trivial.
        # We could keep the total count of rows ourselves, by means of triggers,
        # but that seems too complicated and would slow down normal operation
        # (insert/delete etc).
        GET_LEN = 'SELECT COUNT(*) FROM "%s"' % self.tablename
        rows = self.conn.select_one(GET_LEN)[0]
        return rows if rows is not None else 0

    def __bool__(self):
        # No elements is False, otherwise True
        GET_MAX = 'SELECT MAX(ROWID) FROM "%s"' % self.tablename
        m = self.conn.select_one(GET_MAX)[0]
        # Explicit better than implicit and bla bla
        return True if m is not None else False

    def iterkeys(self):
        GET_KEYS = 'SELECT key FROM "%s" ORDER BY rowid' % self.tablename
        for key in self.conn.select(GET_KEYS):
            yield key[0]

    def itervalues(self):
        GET_VALUES = 'SELECT value FROM "%s" ORDER BY rowid' % self.tablename
        for value in self.conn.select(GET_VALUES):
            yield self.decode(value[0])

    def iteritems(self):
        GET_ITEMS = 'SELECT key, value FROM "%s" ORDER BY rowid' % self.tablename
        for key, value in self.conn.select(GET_ITEMS):
            yield key, self.decode(value)

    def keys(self):
        return self.iterkeys() if major_version > 2 else list(self.iterkeys())

    def values(self):
        return self.itervalues() if major_version > 2 else list(self.itervalues())

    def items(self):
        return self.iteritems() if major_version > 2 else list(self.iteritems())

    def __contains__(self, key):
        HAS_ITEM = 'SELECT 1 FROM "%s" WHERE key = ?' % self.tablename
        return self.conn.select_one(HAS_ITEM, (key,)) is not None

    def __getitem__(self, key):
        GET_ITEM = 'SELECT value FROM "%s" WHERE key = ?' % self.tablename
        item = self.conn.select_one(GET_ITEM, (key,))
        if item is None:
            raise KeyError(key)
        return self.decode(item[0])

    def __setitem__(self, key, value):
        if self.flag == 'r':
            raise RuntimeError('Refusing to write to read-only SqliteDict')

        ADD_ITEM = 'REPLACE INTO "%s" (key, value) VALUES (?,?)' % self.tablename
        self.conn.execute(ADD_ITEM, (key, self.encode(value)))

    def __delitem__(self, key):
        if self.flag == 'r':
            raise RuntimeError('Refusing to delete from read-only SqliteDict')

        if key not in self:
            raise KeyError(key)
        DEL_ITEM = 'DELETE FROM "%s" WHERE key = ?' % self.tablename
        self.conn.execute(DEL_ITEM, (key,))

    def update(self, items=(), **kwds):
        if self.flag == 'r':
            raise RuntimeError('Refusing to update read-only SqliteDict')

        try:
            items = items.items()
        except AttributeError:
            pass
        items = [(k, self.encode(v)) for k, v in items]

        UPDATE_ITEMS = 'REPLACE INTO "%s" (key, value) VALUES (?, ?)' % self.tablename
        self.conn.executemany(UPDATE_ITEMS, items)
        if kwds:
            self.update(kwds)

    def __iter__(self):
        return self.iterkeys()

    def clear(self):
        if self.flag == 'r':
            raise RuntimeError('Refusing to clear read-only SqliteDict')

        CLEAR_ALL = 'DELETE FROM "%s";' % self.tablename  # avoid VACUUM, as it gives "OperationalError: database schema has changed"
        self.conn.commit()
        self.conn.execute(CLEAR_ALL)
        self.conn.commit()

    def commit(self, blocking=True):
        """
        Persist all data to disk.

        When `blocking` is False, the commit command is queued, but the data is
        not guaranteed persisted (default implication when autocommit=True).
        """
        if self.conn is not None:
            self.conn.commit(blocking)
    sync = commit

    def close(self, do_log=True, force=False):
        if do_log:
            logger.debug("closing %s" % self)
        if hasattr(self, 'conn') and self.conn is not None:
            if self.conn.autocommit and not force:
                # typically calls to commit are non-blocking when autocommit is
                # used.  However, we need to block on close() to ensure any
                # awaiting exceptions are handled and that all data is
                # persisted to disk before returning.
                self.conn.commit(blocking=True)
            self.conn.close(force=force)
            self.conn = None
        if self.in_temp:
            try:
                os.remove(self.filename)
            except:
                pass

    def terminate(self):
        """Delete the underlying database file. Use with care."""
        if self.flag == 'r':
            raise RuntimeError('Refusing to terminate read-only SqliteDict')

        self.close()

        if self.filename == ':memory:':
            return

        logger.info("deleting %s" % self.filename)
        try:
            if os.path.isfile(self.filename):
                os.remove(self.filename)
        except (OSError, IOError):
            logger.exception("failed to delete %s" % (self.filename))

    def __del__(self):
        # like close(), but assume globals are gone by now (do not log!)
        try:
            self.close(do_log=False, force=True)
        except Exception:
            # prevent error log flood in case of multiple SqliteDicts
            # closed after connection lost (exceptions are always ignored
            # in __del__ method.
            pass

# Adding extra methods for python 2 compatibility (at import time)
if major_version == 2:
    SqliteDict.__nonzero__ = SqliteDict.__bool__
    del SqliteDict.__bool__  # not needed and confusing
#endclass SqliteDict


class SqliteMultithread(Thread):
    """
    Wrap sqlite connection in a way that allows concurrent requests from multiple threads.

    This is done by internally queueing the requests and processing them sequentially
    in a separate thread (in the same order they arrived).

    """
    def __init__(self, filename, autocommit, journal_mode):
        super(SqliteMultithread, self).__init__()
        self.filename = filename
        self.autocommit = autocommit
        self.journal_mode = journal_mode
        # use request queue of unlimited size
        self.reqs = Queue()
        self.setDaemon(True)  # python2.5-compatible
        self.exception = None
        self.log = logging.getLogger('sqlitedict.SqliteMultithread')
        self.start()

    def run(self):
        if self.autocommit:
            conn = sqlite3.connect(self.filename, isolation_level=None, check_same_thread=False)
        else:
            conn = sqlite3.connect(self.filename, check_same_thread=False)
        conn.execute('PRAGMA journal_mode = %s' % self.journal_mode)
        conn.text_factory = str
        cursor = conn.cursor()
        conn.commit()
        cursor.execute('PRAGMA synchronous=OFF')

        res = None
        while True:
            req, arg, res, outer_stack = self.reqs.get()
            if req == '--close--':
                assert res, ('--close-- without return queue', res)
                break
            elif req == '--commit--':
                conn.commit()
                if res:
                    res.put('--no more--')
            else:
                try:
                    cursor.execute(req, arg)
                except Exception as err:
                    self.exception = (e_type, e_value, e_tb) = sys.exc_info()
                    inner_stack = traceback.extract_stack()

                    # An exception occurred in our thread, but we may not
                    # immediately able to throw it in our calling thread, if it has
                    # no return `res` queue: log as level ERROR both the inner and
                    # outer exception immediately.
                    #
                    # Any iteration of res.get() or any next call will detect the
                    # inner exception and re-raise it in the calling Thread; though
                    # it may be confusing to see an exception for an unrelated
                    # statement, an ERROR log statement from the 'sqlitedict.*'
                    # namespace contains the original outer stack location.
                    self.log.error('Inner exception:')
                    for item in traceback.format_list(inner_stack):
                        self.log.error(item)
                    self.log.error('')  # deliniate traceback & exception w/blank line
                    for item in traceback.format_exception_only(e_type, e_value):
                        self.log.error(item)

                    self.log.error('')  # exception & outer stack w/blank line
                    self.log.error('Outer stack:')
                    for item in traceback.format_list(outer_stack):
                        self.log.error(item)
                    self.log.error('Exception will be re-raised at next call.')

                if res:
                    for rec in cursor:
                        res.put(rec)
                    res.put('--no more--')

                if self.autocommit:
                    conn.commit()

        self.log.debug('received: %s, send: --no more--', req)
        conn.close()
        res.put('--no more--')

    def check_raise_error(self):
        """
        Check for and raise exception for any previous sqlite query.

        For the `execute*` family of method calls, such calls are non-blocking and any
        exception raised in the thread cannot be handled by the calling Thread (usually
        MainThread).  This method is called on `close`, and prior to any subsequent
        calls to the `execute*` methods to check for and raise an exception in a
        previous call to the MainThread.
        """
        if self.exception:
            e_type, e_value, e_tb = self.exception

            # clear self.exception, if the caller decides to handle such
            # exception, we should not repeatedly re-raise it.
            self.exception = None

            self.log.error('An exception occurred from a previous statement, view '
                           'the logging namespace "sqlitedict" for outer stack.')

            # The third argument to raise is the traceback object, and it is
            # substituted instead of the current location as the place where
            # the exception occurred, this is so that when using debuggers such
            # as `pdb', or simply evaluating the naturally raised traceback, we
            # retain the original (inner) location of where the exception
            # occurred.
            reraise(e_type, e_value, e_tb)

    def execute(self, req, arg=None, res=None):
        """
        `execute` calls are non-blocking: just queue up the request and return immediately.
        """
        self.check_raise_error()

        # NOTE: This might be a lot of information to pump into an input
        # queue, affecting performance.  I've also seen earlier versions of
        # jython take a severe performance impact for throwing exceptions
        # so often.
        stack = traceback.extract_stack()[:-1]
        self.reqs.put((req, arg or tuple(), res, stack))

    def executemany(self, req, items):
        for item in items:
            self.execute(req, item)
        self.check_raise_error()

    def select(self, req, arg=None):
        """
        Unlike sqlite's native select, this select doesn't handle iteration efficiently.

        The result of `select` starts filling up with values as soon as the
        request is dequeued, and although you can iterate over the result normally
        (`for res in self.select(): ...`), the entire result will be in memory.
        """
        res = Queue()  # results of the select will appear as items in this queue
        self.execute(req, arg, res)
        while True:
            rec = res.get()
            self.check_raise_error()
            if rec == '--no more--':
                break
            yield rec

    def select_one(self, req, arg=None):
        """Return only the first row of the SELECT, or None if there are no matching rows."""
        try:
            return next(iter(self.select(req, arg)))
        except StopIteration:
            return None

    def commit(self, blocking=True):
        if blocking:
            # by default, we await completion of commit() unless
            # blocking=False.  This ensures any available exceptions for any
            # previous statement are thrown before returning, and that the
            # data has actually persisted to disk!
            self.select_one('--commit--')
        else:
            # otherwise, we fire and forget as usual.
            self.execute('--commit--')

    def close(self, force=False):
        if force:
            # If a SqliteDict is being killed or garbage-collected, then select_one()
            # could hang forever because run() might already have exited and therefore
            # can't process the request. Instead, push the close command to the requests
            # queue directly. If run() is still alive, it will exit gracefully. If not,
            # then there's nothing we can do anyway.
            self.reqs.put(('--close--', None, Queue(), None))
        else:
            # we abuse 'select' to "iter" over a "--close--" statement so that we
            # can confirm the completion of close before joining the thread and
            # returning (by semaphore '--no more--'
            self.select_one('--close--')
            self.join()
#endclass SqliteMultithread


In [140]:
"""
>>> from sqlitedict import SqliteDict
>>> mydict = SqliteDict('./my_db.sqlite', autocommit=True)
>>> mydict['some_key'] = any_picklable_object
>>> print mydict['some_key']  # prints the new value
>>> for key, value in mydict.iteritems():
>>>     print key, value
>>> print len(mydict) # etc... all dict functions work
>>> mydict.close()
"""
mydict = SqliteDict('./scores.sqlite', autocommit=True)

In [141]:
import sqlite3
from sqlite3 import Error

"""
CREATE TABLE `scores` (
	`key`	TEXT,
	`value`	REAL,
	PRIMARY KEY(`key`)
)
"""
sqlite_file = '/Users/jdgibson/git/QueueScheduler2.0/scores.sqlite' 
table = "scores"
 


In [142]:
# create a database connection
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())
conn.close()

[('scores',)]


In [143]:
def get_score(conn, key):
    """

    """
    c = conn.cursor()
    c.execute("SELECT * FROM scores WHERE key='{}'".format(key))
    # conn.close()  
    row = c.fetchone()
    return row

In [144]:
def set_score(conn, key, value):
    """

    """
    c = conn.cursor()
    
    # A) Inserts an ID with a specific value in a second column
    try:
        sql = "REPLACE INTO scores VALUES ('{}', {})".format(key, value)
        if True:
            print("sql: ", sql)
        
        # Can do this with new versions of sqlite.  It does commits automatically.
        with conn:
            conn.execute(sql)

        # c.execute(sql)
        # Save (commit) the changes
        # conn.commit()
        # conn.close()
    except sqlite3.IntegrityError:
        print('ERROR: ID already exists in PRIMARY KEY column {}'.format("key"))


In [145]:
def get_key(constraint, target, time):
    """
    key = "{}.{}.{}".format(target, 
                            str(time).replace(" ","_"), 
                            constraint)
    """
    # Using terrestial time (tt)
    time.format = 'isot'
    key = "{}_{}_{}".format(target, 
                            str(time), 
                            constraint)
    key.replace(" ", "")
    if True:
        print("key:", key)
    return key

In [146]:
def dict_factory(cursor, row):
    d = {}
    for idx,col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [147]:
mmto = Observer(longitude=249.11499999999998*u.deg,
                                 latitude=31.688333333333333*u.deg, 
                                 elevation=2608*u.m,
                                 name="mmto",
                                 timezone="America/Phoenix")
times = Time(["2015-08-01 06:00", "2015-08-01 12:00"])

In [148]:
print(mmto)

<Observer: name='mmto',
    location (lon, lat, el)=(-110.88500000000002 deg, 31.688333333333325 deg, 2607.999999999073 m),
    timezone=<DstTzInfo 'America/Phoenix' LMT-1 day, 16:32:00 STD>>


In [149]:
# Read in the table of targets
from astropy.io import ascii
target_table = ascii.read('targets.txt')

In [150]:
# Create astroplan.FixedTarget objects for each one in the table
targets = [FixedTarget(coord=SkyCoord(ra=ra*u.deg, dec=dec*u.deg), name=name)
           for name, ra, dec in target_table]

In [151]:
print(targets)

[<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>, <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>, <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>, <FixedTarget "Algol" at SkyCoord (ICRS): (ra, dec) in deg ( 47.04221855,  40.95564667)>, <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>, <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>]


In [152]:
k = get_key("MeridianConstraint", targets[0].name, times[0])
if True:
    print("k = ", k)

key: Polaris_2015-08-01T06:00:00.000_MeridianConstraint
k =  Polaris_2015-08-01T06:00:00.000_MeridianConstraint


In [153]:
conn = sqlite3.connect(sqlite_file)
conn.row_factory = dict_factory
value = 0.3424
# conn = sqlite3.connect(sqlite_file, timeout=10)
set_score(conn, k, value)

sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T06:00:00.000_MeridianConstraint', 0.3424)


In [154]:
# conn = sqlite3.connect(sqlite_file, timeout=10)
s = get_score(conn, k)
print("score = ", repr(s))

score =  {'key': 'Polaris_2015-08-01T06:00:00.000_MeridianConstraint', 'value': 0.3424}


In [155]:
# Define an astroplan constraint for the distance of the target from the meridian.
# The returned value is either a boolean [0,1] if the target is outside of an allowed time
# from meridian transit or a float from [0.0:1.0], where the value is 1.0
# when the target is on the meridian to 0.0 when it is at the anti-meridian (12 hours from the meridian)
class MeridianConstraint(Constraint):
    """Constrains the time for targets from meridian transit.

    Principal investigators (PI's) are required to assigned an integer priority from 1 (highest) to 3 (lowest) to each of their targets.
    The targets should be equally divided into the three priorities (i.e., 1, 2, and 3) so that 1/3 of the requested time correspondes to each
    priority.
    This equal division into the three priorities by even time requested is needed to keep scheduling fair for all projects.
    Every effort will be made to observe all targets, but PI's should anticipate that at least part of their priority 3 targets will not be observed because of poor weather or other causes.


    """
    def __init__(self, mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=False, 
                 debug=False):
        """
        Parameters
        ----------
        max : `~astropy.units.Quantity` or `None` (optional)
            Maximum acceptable separation (in decimal hours) between meridian and target (inclusive).
            `None` indicates no constraint of how far the target can be from the meridian.
        boolean_constraint : bool
            If True, the constraint is treated as a boolean (True for within the
            limits and False for outside).  If False, the constraint returns a
            float on [0, 1], where 0 is when the target is on the anti-meridian and
            1 is when the target is on the meridian.
        """
        self.mode = mode 
        self.min_alt_degrees = min_alt_degrees
        self.max_solar_altitude = max_solar_altitude
        self.grid_times_targets = grid_times_targets
        self.debug = debug
        

    def compute_constraint(self, times, observer, targets):
        """
        The MeridianConstraint is calculated by: 1) determining the number of hours the target is from the meridian, and 2) calculating a constraint using Math.abs((12. - hours_from_meridan)/12.0) for the target's position at the beginning, middle, and end of the observing block.  
        The calculated scores for these three times will be different.  
        This causes the constraint to equal 1.0 on the meridian and 0.0 on the anti-meridian (12 hours away).  
        Since the absolute value is used, it doesn't matter which direction the target is from the meridan, i.e., positive hours or negative hours.  Values will always vary from 1.0 to 0.0
        It is possible that the target passes through the meridian during the observing block, i.e., it "transits". 
        Caution should be used in cases where the target transits in that azimuth velocities can be very large if the target is close to zenith.
        The maximum AltitudeConstraint should help prevent extremely large azimuth velocities.  

        It should be remembered that constraint scores are calculated at the beginning, middle, and end of each observing block as part of score for the block.
        This causes the constraint to be multiplied by itself three times and the constraint to vary as 1/X^^3 rather than 1/X.

        """
        
        # 12 hours: the maximum possible time for a target to be from the meridian 
        seconds_in_12hrs = 43200     # 12 hours ==> 12 * 60 * 60 = 43200 seconds
        
        # Set up to TimeDelta constatnt values for future use.
        dt_0hrs = TimeDelta(0,format='sec')
        dt_1hrs = TimeDelta(3600,format='sec')
        dt_1_5hrs = TimeDelta(3600*1.5,format='sec')
        dt_2hrs = TimeDelta(3600*2.0,format='sec')
        dt_2_5hrs = TimeDelta(3600*2.5,format='sec')
        dt_3hrs = TimeDelta(3600*3.0,format='sec')
        dt_3_5hrs = TimeDelta(3600*3.5,format='sec')
        dt_4hrs = TimeDelta(3600*4.0,format='sec')
                
        
        
        # This list will eventually be of length = len(targets) * len(times).
        # We'll reshape it when done.
        # To do:  See if there is a way to user more numpy functions for these
        # calculations to speed things up.
        mask=[]
        
        for target in targets:
            print("target: ", target)
            """
            if False:
                # This is a short-circuit to speed up computation.
                #
                # If the altitude is below the minimum allowed altitude,
                # assign zero's to all times for this target.
                # 
                # In reality, the scores for one or more times within the 
                # observing block could be greater than zero, but
                # the constraint will fail overall for the target since 
                # there is at least one zero score.
            
                cached_altaz = _get_altaz(times, observer, target)
                alt = cached_altaz['altaz'].alt
                
                # Step 1: Check if altitudes are below the minimum allowed
                #       altitude.  This allows short-circuiting of the constraint
                #       calculation to improve efficiency.
                #       If the target is below the minimum altitude,
                #       the overall score for the target will be zero anyway.
                alt_check = alt < self.min_alt_degrees

                # Step 2: This is the special case where the target is below 
                # the lower allowed altitude at any time of the time slot.  
                # The scores for all times for this target will be set to zero.
                if alt_check.any() == True:
                    if self.debug:
                        print("alt is less than minimum: %s, %s" % (alt.degree, self.min_alt_degrees))
                    
                    # Adding a list of zeros of length = len(times)
                    mask += [0.0] * len(times)
            
            # Otherwise, we need to look in detail for each time.
            else:
            """
            if True:
                for time in times:
                    constraint_name = type(self).__name__
                    key = get_key(constraint_name, target.name, time)
                    obj = get_score(conn,key)
                    if obj is not None:
                        score = obj['value']
                        if True:
                            print("Using saved score.", score)
                    else:    
                    
                        # We do a series of tests to see if the observing block is 
                        # is setting early in the evening.  We want to give high
                        # priority to PIPriority == 1 observing blocks that are
                        # setting near sunset and that can still be observed.
                        #
                        # We take into account the duration of observing blocks
                        # when doing this special "sunset" mode.
                        #
                        # Step 3:  Get the time of the previous sunset.
                        #       This will be used to see if the time is close to sunset.
                        prev_sun_set_time = observer.sun_set_time(time, 
                                            which='previous', 
                                            horizon=self.max_solar_altitude)

                        # Step 4: Calculate the time from the previous sunset.  
                        #        This is an indication of how close we are to sunset.
                        #        It will be a small number if we are trying to observe
                        #        just after sunset.
                        td1 = time - prev_sun_set_time 

                        # Step 5: Get the time of the next target rise.
                        #       This will be used to see if the target is close to rising
                        #       in the east in the morning.
                        next_target_set = observer.target_set_time(time,target,
                                            which="next", 
                                            horizon=self.min_alt_degrees )

                        # Step 6: Calculate the time from the next target setting.
                        #        This number will be a small positive number when 
                        #        the target is above the western horizon.
                        td2 = next_target_set - time

                        # if False and verbose:
                        #    print("tx1: {}, sec: {}, tx2: {}, sec: {}".format(tx1, tx1.sec, tx2, tx2.sec))

                        # Note: The next three steps use a "sunset" mode where we want to give
                        #       high priority to targets that will be setting within the next 2-4
                        #       hours.  We use a graded approach for scoring, based on the duration
                        #       of the target/observing block.

                        # Step 8: This is the "2-hour-target-duration" sunset special case.  
                        #       Evaluate for targets/observing blocks that are more the
                        #       _2_ hours in duration, and we are within _4_ hours after sunset,
                        #       and the target will be setting within _4_ hours.
                        #       Only do this for priority 1 targets.
                        #       This is our only chance to observe them.
                        #       The score is set to 1.0 to give it the maximum chance of being observed.

                        if self.mode == "sunset" and \
                                td1 <= dt_4hrs and td1 > dt_0hrs and \
                                td2 <= dt_4hrs and td2 > dt_0hrs and \
                                target.duration >= 2.0 * u.hour and \
                                target.pi_priority == 1.0:
                            score = 1.0
                            if verbose:
                                print("Sunset special case (>= 2-hr duration), score:",score)

                        # Step 9: This is the "1-hour-target-duration" sunset special case.  
                        #       Evaluate for targets/observing blocks that are 1-2 hours
                        #       in duration, and we are within _3_ hours after sunset,
                        #       and the target will be setting within _3_ hours.
                        #       Only do this for priority 1 targets.
                        #       This is our only chance to observe them.
                        #       The score is set to 1.0 to give it the maximum chance of being observed.
                        elif self.mode == "sunset" and \
                                td1 <= dt_3hrs and td1 > dt_0hrs and \
                                td2 <= dt_3hrs and td2 > dt_0hrs and \
                                target.duration >= 1.0 * u.hour and \
                                target.pi_priority == 1.0:
                            score = 1.0
                            if verbose:
                                print("Sunset special case, (>= 1-hour and < 2-hour duration) score:",score)

                        # Step 10: This is the "<1-hour-target-duration" sunset special case.  
                        #       Evaluate for targets/observing blocks that are <1 hour
                        #       in duration, and we are within _2_ hours after sunset,
                        #       and the target will be setting within _2_ hours.
                        #       Only do this for priority 1 targets.
                        #       This is our only chance to observe them.
                        #       The score is set to 1.0 to give it the maximum chance of being observed.
                        elif self.mode == "sunset" and \
                                td1 <= dt_2hrs and td1 > dt_0hrs and \
                                td2 <= dt_2hrs and td2 > dt_0hrs and \
                                target.pi_priority == 1.0:
                            score = 1.0
                            if verbose:
                                print("Sunset special case, (any duration) score:",score)

                        # Step 11:  If all of the other conditions have not been true,
                        #       Determine how far the target is from the meridian in seconds
                        #       and divide by 12 hours (== 43200 seconds) 
                        #       The target can be in either rising towards the meridian or
                        #       setting away from the meridian.
                        else:

                            meridian_time = observer.target_meridian_transit_time(time,target,which='nearest')
                            diff = abs(time.unix - meridian_time.unix)

                            # There are times when the meridian time is 24 hours off.
                            # So, the math here accounts for that.
                            # If the time difference is more than 24 hours (43200 seconds), 
                            # subtract 24 hours.
                            if diff > 43200:
                                diff -= 86400 # 24 hours in seconds.
                                # Recheck that we are using an absolute value.
                                diff = abs(diff)

                            # Here is the meridian scoring algorithm.  
                            # The closer to the meridian the closer the score is to one.  
                            # The range of scores is 1.0 (on the meridian) to 
                            # 0.0 (on the anti-meridian).
                            score = 1.0 - (diff / 43200.0)               
                            # Shouldn't need these, but just checking.
                            # The score should already range from 0.0 to 1.0.
                            if score < 0.0:
                                score = 0.0
                            if score > 1.0:
                                score = 1.0
                    
                    # Add the new score to the 1-D list.
                    set_score(conn,key,score)
                    mask.append(score)
            
        if self.debug:
            print("mask")
            print(repr(mask))
            
        # Turn the mask into a numpy array and reshape.
        a = np.reshape(np.array(mask),[len(targets), len(times)])
            
        if self.debug:
            print("a")
            print(repr(a))
        
        return a

In [156]:
m = MeridianConstraint( mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=False, 
                 debug=True)

In [157]:
print("Constraint name: ", repr(type(m).__name__))

Constraint name:  'MeridianConstraint'


In [158]:
m.compute_constraint(times, mmto, targets)

target:  <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>
key: Polaris_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.3424
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T06:00:00.000_MeridianConstraint', 0.3424)
key: Polaris_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T12:00:00.000_MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T06:00:00.000_MeridianConstraint', 0.9485614448785782)
key: Vega_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T12:00:00.000_MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" 

array([[ 0.3424    ,  0.86720344],
       [ 0.94856144,  0.44861741],
       [ 0.97659309,  0.52320695],
       [ 0.34264003,  0.84261654],
       [ 0.16800772,  0.66800573],
       [ 0.24455534,  0.26091625]])

In [159]:
print("targets: ", repr(targets))

targets:  [<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>, <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>, <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>, <FixedTarget "Algol" at SkyCoord (ICRS): (ra, dec) in deg ( 47.04221855,  40.95564667)>, <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>, <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>]


In [160]:
m = MeridianConstraint( mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=False, 
                 debug=True)
start = datetime.datetime.now()
for count in range(10):
    m.compute_constraint(times, mmto, targets)
end = datetime.datetime.now()
print("Elapsed time: ", str(end - start))

target:  <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>
key: Polaris_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.3424
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T06:00:00.000_MeridianConstraint', 0.3424)
key: Polaris_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T12:00:00.000_MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T06:00:00.000_MeridianConstraint', 0.9485614448785782)
key: Vega_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T12:00:00.000_MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" 

key: Rigel_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.668005728158686
sql:  REPLACE INTO scores VALUES ('Rigel_2015-08-01T12:00:00.000_MeridianConstraint', 0.668005728158686)
target:  <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>
key: Regulus_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.24455533642459804
sql:  REPLACE INTO scores VALUES ('Regulus_2015-08-01T06:00:00.000_MeridianConstraint', 0.24455533642459804)
key: Regulus_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.2609162507564933
sql:  REPLACE INTO scores VALUES ('Regulus_2015-08-01T12:00:00.000_MeridianConstraint', 0.2609162507564933)
mask
[0.3424, 0.8672034377190802, 0.9485614448785782, 0.4486174117801366, 0.9765930855605337, 0.5232069483068253, 0.3426400301964194, 0.842616540375683, 0.16800771560381955, 0.668005728158686, 0.24455533642459804, 0.2609162507564933]
a
array([[ 0.3424    ,  0.86720344],
       [ 0.94856144,  0.448

key: Polaris_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.8672034377190802
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T12:00:00.000_MeridianConstraint', 0.8672034377190802)
target:  <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>
key: Vega_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.9485614448785782
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T06:00:00.000_MeridianConstraint', 0.9485614448785782)
key: Vega_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.4486174117801366
sql:  REPLACE INTO scores VALUES ('Vega_2015-08-01T12:00:00.000_MeridianConstraint', 0.4486174117801366)
target:  <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>
key: Albireo_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.9765930855605337
sql:  REPLACE INTO scores VALUES ('Albireo_2015-08-01T06:00:00.000_MeridianConstraint', 0.9765930855605337)
key: Al

key: Regulus_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score. 0.2609162507564933
sql:  REPLACE INTO scores VALUES ('Regulus_2015-08-01T12:00:00.000_MeridianConstraint', 0.2609162507564933)
mask
[0.3424, 0.8672034377190802, 0.9485614448785782, 0.4486174117801366, 0.9765930855605337, 0.5232069483068253, 0.3426400301964194, 0.842616540375683, 0.16800771560381955, 0.668005728158686, 0.24455533642459804, 0.2609162507564933]
a
array([[ 0.3424    ,  0.86720344],
       [ 0.94856144,  0.44861741],
       [ 0.97659309,  0.52320695],
       [ 0.34264003,  0.84261654],
       [ 0.16800772,  0.66800573],
       [ 0.24455534,  0.26091625]])
target:  <FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>
key: Polaris_2015-08-01T06:00:00.000_MeridianConstraint
Using saved score. 0.3424
sql:  REPLACE INTO scores VALUES ('Polaris_2015-08-01T06:00:00.000_MeridianConstraint', 0.3424)
key: Polaris_2015-08-01T12:00:00.000_MeridianConstraint
Using saved score.

In [163]:
aakey = _make_cache_key(times, targets)
repr(aakey)

'(2457235.75, 2457236.0, 2, [<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>, <FixedTarget "Vega" at SkyCoord (ICRS): (ra, dec) in deg ( 279.23473479,  38.78368896)>, <FixedTarget "Albireo" at SkyCoord (ICRS): (ra, dec) in deg ( 292.68033548,  27.95968007)>, <FixedTarget "Algol" at SkyCoord (ICRS): (ra, dec) in deg ( 47.04221855,  40.95564667)>, <FixedTarget "Rigel" at SkyCoord (ICRS): (ra, dec) in deg ( 78.63446707, -8.20163837)>, <FixedTarget "Regulus" at SkyCoord (ICRS): (ra, dec) in deg ( 152.09296244,  11.96720878)>])'

In [133]:
repr(_get_meridian_transit_times(times, mmto, targets))

TypeError: unhashable type: 'list'

In [171]:
time = times[0]
target = targets[0]
repr(time)
repr(target)




'<FixedTarget "Polaris" at SkyCoord (ICRS): (ra, dec) in deg ( 37.95456067,  89.26410897)>'

In [ ]:
meridian_transit_times = Observer.target_meridian_transit_time(time,target,which='nearest')

In [ ]:
Observer._meridian_transit_cache[aakey] = dict(times=meridian_transit_times)

In [ ]:

mydict['_meridian_transit_cache'] = _get_meridian_transit_times(times, mmto, targets)